# Processing passive seismic data

This notebook deals with data processing using a passive seismic dataset using [ObsPy](https://docs.obspy.org/).

## It is intended to be run in Colab.


#### Dependencies: Obspy, Numpy, Matplotlib

#### Reset the Jupyter/IPython notebook in order to run it again, press:
***Kernel*** -> ***Restart & Clear Output***

---
## Install Conda and create an *environment*, and install the required *packages* for the work.

* The `condacolab` commands need to be run as the first Code cell!
* After `condacolab.install()`, the Python kernel will be restarted.
* Do not use the `Run all` option. Run the `condacolab` cell _individually_ and wait for the kernel to restart.
* You can only use the `base` environment. Do not try to create new ones; instead update `base` with either:
  * `conda install <packages>`
  * `conda env update -n base -f environment.yml`

## ***Attn:*** Every time you quit or reset the runtime, you wil need to install Conda again.

In [1]:
# Import and install condacolab python library.
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:23
🔁 Restarting kernel...


In [1]:
#  kernel restart HERE!
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [2]:
# Verify the Conda installation. Update.
!conda --version
#!conda update -n base -c defaults conda --yes
!conda update -n base -c conda-forge conda --yes
!conda --version

# Install the certifi package, which is a dependency of the cartopy package, that obspy depends on.
!conda install -f certifi cartopy

# Create environment.yml
!conda install pandas numpy matplotlib numpy>=1.15.0 scipy>=1.4.0 obspy utm segyio

# Locate the Conda package. List environments. Verify the Conda installation.
!conda info
!which conda
!conda env list

# Importing specialized ObsPy packages
from obspy import read
from obspy import UTCDateTime
from obspy import read, Stream

# matplotlib magic command for Colab
%matplotlib inline
# matplotlib magic command for local Notebook
#%matplotlib notebook

# Import matplotlib and alias it as 'plt'
from matplotlib import pyplot as plt
%matplotlib notebook


conda 23.11.0
Channels:
 - conda-forge
Platform: linux-64
Solving environment: \ | / - \ | / done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.7.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

conda 23.11.0


         See 'conda install --help' for details about the --force-reinstall
         and --clobber flags.


Channels:
 - conda-forge
Platform: linux-64
Solving environment: \ | / - \ | / - done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.7.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cartopy
    - certifi





Preparing transaction: | done
Verifying transaction: - done
Executing transaction: | / - \ | / done


=

---
### Accessing a local data

We work with data form an event occured in Acre in 2024 in miniSEED format and already with the instrument response removed [wb11793085_ir.mseed](https://github.com/jandyr/ProSeisSN_Nbk/blob/main/wb11793085_ir.mseed).

|EventID | Time | Lat | Long | Depth(km) | Mww | Location |
| :- | :-: | :- | :- | :-: | :-: | :- |
|11793085|2024-01-20T21:31|-7.2879|-71.464|607|6.6|WESTERN BRAZIL|

As recorded at station IU HKT, distant $44^{\circ}$ from the event.

|Station|Seismometer| Lat | Long | Depth(m) | Channels |
| :- | :- | :- | :- | :-: | :-: |
|HKT: Hockley1 Texas|Streckeisen STS-6A VBB |29.96|-95.84|93|BH1, BH2, BHZ|


Use the read function with the path to the data file.

**st** is a stream object, which contains the data and associated metadata. The data are collections of trace objects.

In [4]:
st = read('./wb11793085_ir.mseed')
print(st)

FileNotFoundError: [Errno 2] No such file or directory: './wb11793085_ir.mseed'

---
### The frequency content

The [spectrogram](https://docs.obspy.org/tutorial/code_snippets/plotting_spectrograms.html) shows the power in frequency bands (along the y-axis) for every few samples in time (along the x-axis). Warmer colours indicate more power - this station has most of it's power below 1 Hz, which is common for a time-period dominated by seismic noise.

In [ ]:
# Read a trace from the stream object.
tr = st[2]
print(tr)
# Print the trace metadata, e.g., tr.stats.npts
print(tr.stats)

# Construct a figure for BHZ trace
fig, ax = plt.subplots()
_ = st[2].spectrogram(log=True, dbscale=True, axes=ax, show=True, cmap = "jet")

plt.show()

## 3) Filtering
**ObsPy** has several [filters](https://docs.obspy.org/packages/autogen/obspy.core.trace.Trace.filter.html).:
- "bandpass" filters *retain* a frequency range $\Delta f=\left[f_{1},f_{2}\right]=f_{1}\leq\Delta f\leq f_{2}$;
- "lowpass" filters *retain* the frequency range $\Delta f\leq f_{cutoff}$;
- "highpass" filters *retain* the frequency range $\Delta f\geq f_{cutoff}$;
- "notch" filters *remove* the frequency range $f_{1}\leq\Delta f\leq f_{2}$, therefore does the *opposite* of bandpass.



In [ ]:
# Import obspy Stream
from obspy import Stream

# BHZ trace. Check.
# Read a trace from the stream object.
tr = st[2]
print(tr.stats.channel)
tr.plot()

# corners=filter order; zerophase=True, results in a zero phase shift.
#tr.filter("highpass", freq=6.)
tr.filter("bandpass", freqmin=6., freqmax=11.)

# Plot filtered trace
tr.plot(color = 'blue');

# Plot spectograms
fig, ax = plt.subplots()
_ = tr.spectrogram(log=True, dbscale=True, axes=ax, show=True, cmap = "jet")